In [1]:
import pandas as pd

df = pd.read_json("../../datas/nuclei/detected.json", lines=True)
# df = pd.read_json("../detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 47064}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:50:40.532834,NaN
1,HEAD,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58566}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:44.945677,NaN
2,GET,/servlet/codesettree?flag=c&status=1&codesetid...,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58590}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.022008,NaN
3,POST,/WEB_VMS/LEVEL15/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58584}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.028476,{}
4,POST,/zentao/user-login.html,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",4d001351-3ab5-4ebe-b891-97c5ef3d1bac,"{'ip': '172.18.0.1', 'port': 58654}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-17 11:51:45.096028,"{'account': 'admin' and updatexml(1,concat(0x..."


In [2]:
attacks_matrix = []
tmp = []
sess_uuids_attacks = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  detection = row.get("response_msg", {}).get("response", {}).get("message", {}).get("detection", {})
  attack_type = detection.get("name", "")

  if sess_uuid in sess_uuids:
    sess_uuids_attacks[sess_uuid].append(attack_type)
  else:
    sess_uuids_attacks[sess_uuid] = [attack_type]
    sess_uuids.append(sess_uuid)

attacks_matrix = list(sess_uuids_attacks.values())
# for paths in sess_uuids_attacks.values():
#   attacks_matrix.append(paths)

sess_uuids_attacks
# attacks_matrix
# print(len(attacks_matrix), len(sess_uuids_paths.keys()))

{'5c03d747-efc4-417c-9ab2-87a504285fb5': ['index'],
 'a7eb06e3-388a-436c-af5c-1778a2b34e3d': ['index',
  'xss',
  'index',
  'index',
  'unknown',
  'wp-content',
  'index',
  'index'],
 '481b180f-6499-4636-a881-d1b5948991e4': ['index',
  'index',
  'index',
  'index',
  'unknown',
  'index',
  'index',
  'index',
  'index',
  'index',
  'wp-content'],
 'aa449b70-96de-4a22-8bbb-816a1491958e': ['unknown',
  'cmd_exec',
  'unknown',
  'index',
  'index',
  'lfi',
  'index',
  'unknown'],
 'e9c12af5-5288-43d8-a728-60deda9557dd': ['sqli',
  'unknown',
  'sqli',
  'index',
  'index',
  'index',
  'index',
  'unknown',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index'],
 'eb6f4177-e8a3-4a3e-a797-9d3b250d9b01': ['unknown', 'index', 'index'],
 'a7da2687-0745-4aa9-885d-d4d76766904d': ['sqli',
  'cmd_exec',
  'unknown',
  'cmd_exec',
  'index',
  'index'],
 'a969159d-11d9-4466-b89b-b3cc8da3df27': ['index',
  'unknown',
  'unknown',
  'sqli',
  'xss',
  'unknown',
  'index',
  'ind

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(attacks_matrix).transform(attacks_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head(10)

,cmd_exec,crlf,index,lfi,rfi,sqli,template_injection,unknown,wp-content,xss
0,False,False,True,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,True,True,True
2,False,False,True,False,False,False,False,True,True,False
3,True,False,True,True,False,False,False,True,False,False
4,False,False,True,False,False,True,False,True,False,False
5,False,False,True,False,False,False,False,True,False,False
6,True,False,True,False,False,True,False,True,False,False
7,False,False,True,False,False,True,False,True,False,True
8,False,False,True,False,False,False,False,True,False,True
9,True,False,True,True,False,False,False,True,False,False


In [7]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True)
frequent_itemsets["length"] = frequent_itemsets["itemsets"].apply(lambda x: len(x))
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets,length
0,0.978425,(index),1
1,0.688242,(unknown),1
2,0.355987,(wp-content),1
3,0.186624,(xss),1
4,0.181230,(lfi),1
...,...,...,...
260,0.002157,"(index, template_injection, lfi, unknown)",4
261,0.001079,"(template_injection, unknown, rfi)",3
262,0.001079,"(index, template_injection, rfi)",3
263,0.001079,"(index, template_injection, unknown, rfi)",4


In [8]:
large_frequent_itemsets = frequent_itemsets[frequent_itemsets["length"] >= 3]
large_frequent_itemsets

,support,itemsets,length
13,0.250270,"(index, unknown, wp-content)",3
18,0.134844,"(index, xss, unknown)",3
19,0.060410,"(index, xss, wp-content)",3
20,0.043150,"(xss, unknown, wp-content)",3
21,0.043150,"(index, xss, unknown, wp-content)",4
...,...,...,...
260,0.002157,"(index, template_injection, lfi, unknown)",4
261,0.001079,"(template_injection, unknown, rfi)",3
262,0.001079,"(index, template_injection, rfi)",3
263,0.001079,"(index, template_injection, unknown, rfi)",4


In [13]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_nuclei", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [10]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [11]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully


In [14]:
# create large data table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types_large (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [15]:
# insert large data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types_large (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in large_frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
